In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import torch.nn as nn
from torchvision.io import read_image
from torch.utils.data import DataLoader
from torchvision.transforms import Resize

from nn_analysis import load_model_for_inference
from dataloader import ImageFolderWithLabels, get_dataset, get_dataloader

from collections import namedtuple

import matplotlib.pyplot as plt

# Lowest-loss images?

Which images in the dataset have the lowest loss?

Investigation using [efficient-donkey-47](https://wandb.ai/ajacobsen/ulc-malaria-autofocus/runs/3at65ww4?workspace=user-ajacobsen)

In [ ]:
# custom dataset

class ImagesAndPaths(ImageFolderWithLabels):
    def __getitem__(self, index):
        path, target = self.samples[index]
        sample = self.loader(path)
        if self.transform is not None:
            sample = self.transform(sample)
        if self.target_transform is not None:
            target = self.target_transform(target)

        return (sample, target), path
    
full_dataset = ImagesAndPaths(
    root = "training_data",
    transform = Resize([150, 200]),
    loader = read_image
)

dataloader = DataLoader(full_dataset, batch_size=1)

In [ ]:
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = load_model_for_inference("trained_models/efficient-donkey-final.pth", dev=dev)

L2 = nn.MSELoss().to(dev)

In [ ]:
import time
t0 = time.perf_counter()

Res = namedtuple("Res", ["pred", "clss", "loss"])
path_to_res = dict()
for i, data in enumerate(dataloader):
    (img, clss), path = data
    img.to(dev)
    clss.to(dev)
    
    with torch.no_grad():
        outputs = net(img).reshape(-1)
        loss = L2(outputs, clss.float())
        path_to_loss[path] = Res(outputs.item(), clss.item(), loss.item())

    if i > 1000:
        break

time.perf_counter() - t0

In [ ]:
path_to_loss